In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# 1. Data Preparation (Synthetic for simplicity)
# Reverse a sequence of characters, e.g., "abc" -> "cba"

In [ ]:
def generate_data(num_samples=10000, max_len=10):
    chars = 'abcdefghijklmnopqrstuvwxyz'
    input_texts = []
    target_texts = []
    for _ in range(num_samples):
        length = np.random.randint(1, max_len + 1)
        input_seq = ''.join(np.random.choice(list(chars), length))
        target_seq = input_seq[::-1]
        input_texts.append(input_seq)
        target_texts.append(target_seq)
    return input_texts, target_texts

input_texts, target_texts = generate_data()

In [3]:
input_texts

['hcttiypvd',
 'hlqjskwyh',
 'hxkjvnp',
 'ykorsuc',
 'hpxcsi',
 'a',
 'vtbfcze',
 'b',
 'hdj',
 'nxgkshfpa',
 'yeit',
 'hgcoq',
 'f',
 'yfd',
 'byrsrj',
 'nzuxonwg',
 'k',
 'jmtyphjz',
 'dsl',
 'rqt',
 'uhpcmuri',
 'vq',
 'ejlmwelbi',
 'lucmpwzjm',
 'ydgesbhm',
 'bqkyiffmn',
 'eg',
 'sxyvxea',
 'qhvmasnyvs',
 'sks',
 'm',
 'oigwpqhtcv',
 'jujg',
 'tx',
 'qs',
 'qlmdgjipik',
 'obyil',
 'lj',
 'nqurlup',
 'vwaaewa',
 'qy',
 'k',
 'v',
 'fszxyg',
 'fbh',
 'j',
 'zaujz',
 'cnonf',
 'xebmquj',
 'jgzcehkqu',
 'navdm',
 'm',
 'zt',
 'jc',
 'bmhkc',
 'kmu',
 'byveyzu',
 'phqzzuzsl',
 'em',
 'p',
 'sf',
 'ifcud',
 'bjiamaa',
 'ylbrpqgvg',
 'sm',
 'gxctptyx',
 'pq',
 'vxlhgcegz',
 'kpuje',
 'xsalowxi',
 'qlzcomudc',
 'tkjduirrez',
 'ddhvtmwtwa',
 'lm',
 'gjs',
 'odthigbwda',
 'ialm',
 'lzapi',
 'mrmhda',
 'g',
 'xrwofyqe',
 'ajy',
 'mvratn',
 'thdjnnungl',
 'lrvxvuw',
 'hqktxsov',
 'dvxywdij',
 'rclhf',
 'lyumg',
 'rvxfymq',
 'tbeut',
 'xgh',
 'hzuqxa',
 'sx',
 'ehpicjxeed',
 'f',
 'l',
 'r',
 '

In [4]:
# Add special tokens and create vocabulary
input_vocab = sorted(list(set("".join(input_texts))))
target_vocab = sorted(list(set("".join(target_texts))))

input_vocab_size = len(input_vocab) + 3  # +3 for <pad>, <start>, and <end>
target_vocab_size = len(target_vocab) + 3  # +3 for <pad>, <start>, and <end>


In [6]:
input_vocab_size, target_vocab_size

(29, 29)